In [23]:
import pandas as pd
from IPython.display import display, HTML
import re

In [38]:
def parse_describe_result(describe_str):
    # Remove the 'DescribeResult(' and trailing ')' parts, then split by commas
    cleaned_str = describe_str.replace('DescribeResult(', '').replace(')', '').replace('(', '').replace(')', '').replace(' ','')
    parts = cleaned_str.split(',')
    min_val = float(parts[1].split('=')[1].strip())
    max_val = float(parts[2])
    mean_val = float(parts[3].split('=')[1].strip())
    variance_val = float(parts[4].split('=')[1].strip())
    return pd.Series({'Min': min_val, 'Max': max_val, 'Mean': mean_val, 'Variance': variance_val})



In [39]:
result_file_directory = "../results/experiment3 20240827-1605.csv"
df = pd.read_csv(result_file_directory)
unique_data = df['Data'].unique()

for data_value in unique_data:
    filtered_df = df[df['Data'] == data_value][["Method","Measure","Value"]]
    table_1_filtered = filtered_df[filtered_df['Measure'].isin(['ambiguity', 'discrepancy'])].pivot(index='Method', columns='Measure', values='Value')
    display(HTML(f"<h5>Table for {data_value} Ambiguity and Discrepancy</h5>"))
    display(HTML(f"{table_1_filtered.to_html()}"))
    display(HTML(f"<h5>Table for {data_value} Variance in Predictions</h5>"))
    table_2 = filtered_df[filtered_df['Measure'] == 'variance in predictions']['Value'].apply(parse_describe_result)
    table_2.index = filtered_df[filtered_df['Measure'] == 'variance in predictions']['Method']
    display(HTML(f"{table_2.to_html()}"))
    display(HTML(f"<h5>Table for {data_value} Disagreements</h5>"))
    table_3 = filtered_df[filtered_df['Measure'] == 'disagreement']['Value'].apply(parse_describe_result)
    table_3.index = filtered_df[filtered_df['Measure'] == 'disagreement']['Method']
    display(HTML(f"{table_3.to_html()}"))


Measure,ambiguity,discrepancy
Method,,
M,0.21,0.136
a,0.23,0.136
b,0.197,0.114
c,0.185,0.117
d,0.145,0.097


,Min,Max,Mean,Variance
Method,,,,
M,0.000407,0.047625,0.005499,0.000052
a,0.000374,0.023252,0.003747,0.000011
b,0.000020,0.022602,0.002709,0.000010
c,0.000417,0.035491,0.003958,0.000017
d,0.000085,0.031793,0.003529,0.000012


,Min,Max,Mean,Variance
Method,,,,
M,0.0,0.090226,0.031146,0.001018
a,0.0,0.091479,0.017809,0.000373
b,0.0,0.091479,0.014160,0.000306
c,0.0,0.058897,0.018956,0.000242
d,0.0,0.037594,0.014536,0.000234


Measure,ambiguity,discrepancy
Method,,
M,0.408,0.309
a,0.368,0.225
b,0.287,0.189
c,0.264,0.169
d,0.154,0.134


,Min,Max,Mean,Variance
Method,,,,
M,0.000826,0.157337,0.019928,0.000443
a,0.000445,0.078187,0.009176,0.000094
b,0.000078,0.077059,0.007284,0.000094
c,0.000331,0.105521,0.009749,0.000121
d,0.000009,0.104731,0.006354,0.000179


,Min,Max,Mean,Variance
Method,,,,
M,0.0,0.352227,0.130870,0.009763
a,0.0,0.268016,0.063720,0.002095
b,0.0,0.183806,0.051690,0.001332
c,0.0,0.159109,0.065171,0.002547
d,0.0,0.094332,0.035975,0.001304


Measure,ambiguity,discrepancy
Method,,
M,0.467,0.365
a,0.335,0.185
b,0.275,0.151
c,0.199,0.146
d,0.194,0.127


,Min,Max,Mean,Variance
Method,,,,
M,0.002104,0.163421,0.025126,0.000207
a,0.000013,0.105181,0.009020,0.000102
b,0.000015,0.088697,0.008069,0.000084
c,0.000013,0.092786,0.007128,0.000084
d,0.000015,0.087859,0.007012,0.000095


,Min,Max,Mean,Variance
Method,,,,
M,0.0,0.404453,0.149117,0.016378
a,0.0,0.173512,0.059188,0.001732
b,0.0,0.173512,0.054427,0.001690
c,0.0,0.132975,0.045641,0.001928
d,0.0,0.107793,0.045580,0.001507
